In [2]:
import pandas as pd
import os
from PIL import Image
import utils as utils

CONFIG_DATA = utils.config_load()

# Create an empty DataFrame to store the image data and labels
df = pd.DataFrame(columns=["Image", "Label"])

# Retrieve the list of files from the SSH server directory
data_directory = CONFIG_DATA['data_source']
file_list = os.listdir(data_directory)

# Resize dimensions
resize_width = 64
resize_height = 64

# Loop through each image file in the SSH server directory
for filename in file_list:
    if filename.endswith(".jpeg"):
        # Load the image using PIL
        image_path = os.path.join(data_directory, filename)
        image = Image.open(image_path)
        
        # Resize the image
        image = image.resize((resize_width, resize_height))

        # Add the class name as a prefix to the label
        if filename.startswith("Banana"):
            label = "Banana"
        elif filename.startswith("Apple"):
            label = "Apple"
        elif filename.startswith("Grape"):
            label = "Grape"

        # Add the image and label to the DataFrame
        df.loc[len(df)] = [image, label]
        # df = df.append({'Image': image, 'Label': label}, ignore_index=True)

# Print the resulting DataFrame
print(df.head())

# Dump File
utils.dump_json(df, CONFIG_DATA['raw_set_path'])


                                               Image  Label
0  <PIL.Image.Image image mode=RGB size=64x64 at ...  Apple
1  <PIL.Image.Image image mode=RGB size=64x64 at ...  Grape
2  <PIL.Image.Image image mode=RGB size=64x64 at ...  Apple
3  <PIL.Image.Image image mode=RGB size=64x64 at ...  Apple
4  <PIL.Image.Image image mode=RGB size=64x64 at ...  Apple


In [3]:
from sklearn.model_selection import train_test_split
import utils as utils

CONFIG_DATA = utils.config_load()

# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=CONFIG_DATA['test_size'], random_state=CONFIG_DATA['random_state'])
train_df, val_df = train_test_split(train_df, test_size=CONFIG_DATA['test_size'], random_state=CONFIG_DATA['random_state'])

import numpy as np
from keras.preprocessing.image import img_to_array

# Convert images to numpy arrays
train_images = np.array([img_to_array(img) for img in train_df['Image']])
val_images = np.array([img_to_array(img) for img in val_df['Image']])
test_images = np.array([img_to_array(img) for img in test_df['Image']])

# Normalize the image pixel values between 0 and 1
train_images = train_images.astype('float32') / 255.0
val_images = val_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Convert labels to categorical
num_classes = df['Label'].nunique()
train_labels = pd.get_dummies(train_df['Label']).values
val_labels = pd.get_dummies(val_df['Label']).values
test_labels = pd.get_dummies(test_df['Label']).values

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=train_images.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
model.fit(train_images, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(val_images, val_labels))

# Evaluate the model on the testing set
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Make predictions on the testing set
predictions = model.predict(test_images)

# Convert the predictions to labels
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(test_labels, axis=1)

# Compare the predicted labels with true labels
correct_predictions = np.sum(predicted_labels == true_labels)
total_predictions = len(test_labels)
accuracy = correct_predictions / total_predictions
print("Accuracy:", accuracy)


# Dump File Model
model.save(CONFIG_DATA['model_path'])

Epoch 1/10


99/99 [==============================] - 7s 65ms/step - loss: 1.0308 - accuracy: 0.4723 - val_loss: 0.8963 - val_accuracy: 0.5879
Epoch 2/10
99/99 [==============================] - 5s 49ms/step - loss: 0.8552 - accuracy: 0.6156 - val_loss: 0.8336 - val_accuracy: 0.6271
Epoch 3/10
99/99 [==============================] - 5s 47ms/step - loss: 0.7677 - accuracy: 0.6595 - val_loss: 0.7816 - val_accuracy: 0.6536
Epoch 4/10
99/99 [==============================] - 5s 46ms/step - loss: 0.7052 - accuracy: 0.7060 - val_loss: 0.7821 - val_accuracy: 0.6700
Epoch 5/10
99/99 [==============================] - 5s 47ms/step - loss: 0.6338 - accuracy: 0.7325 - val_loss: 0.8015 - val_accuracy: 0.6675
Epoch 6/10
99/99 [==============================] - 5s 46ms/step - loss: 0.5581 - accuracy: 0.7834 - val_loss: 0.8507 - val_accuracy: 0.6650
Epoch 7/10
99/99 [==============================] - 5s 47ms/step - loss: 0.4692 - accuracy: 0.8154 - val_loss: 0.8029 - val_accuracy: 0.6675
Epoch 8/10
99/99 [======

/home/aegis_poc/aegis_venv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
